In [5]:
from pathlib import Path
import pandas as pd

project_root = Path.cwd().parent  # go up one level from notebooks/
data_raw = project_root / "data_raw"

orders = pd.read_csv(data_raw / "olist_orders_dataset.csv.zip")
items = pd.read_csv(data_raw / "olist_order_items_dataset.csv.zip")
customers = pd.read_csv(data_raw / "olist_customers_dataset.csv.zip")


In [6]:
orders = orders[["order_id", "customer_id", "order_status", "order_purchase_timestamp"]]
items = items[["order_id", "price"]]
customers = customers[["customer_id", "customer_city", "customer_state"]]


In [7]:
orders["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"])
orders["order_date"] = orders["order_purchase_timestamp"].dt.date
orders["order_month"] = orders["order_purchase_timestamp"].dt.to_period("M").dt.to_timestamp()


In [8]:
order_revenue = (
    items.groupby("order_id", as_index=False)["price"]
    .sum()
    .rename(columns={"price": "order_revenue"})
)


In [9]:
fact_orders = orders.merge(order_revenue, on="order_id", how="left")


In [10]:
dim_customers = customers.drop_duplicates("customer_id").copy()


In [11]:
date_min = fact_orders["order_date"].min()
date_max = fact_orders["order_date"].max()

dim_date = pd.DataFrame({"date": pd.date_range(date_min, date_max, freq="D")})
dim_date["year"] = dim_date["date"].dt.year
dim_date["month"] = dim_date["date"].dt.month
dim_date["year_month"] = dim_date["date"].dt.to_period("M").dt.to_timestamp()


In [13]:
data_processed = Path("../data_processed")
data_processed.mkdir(exist_ok=True)

fact_orders.to_csv(data_processed / "fact_orders.csv", index=False)
dim_customers.to_csv(data_processed / "dim_customers.csv", index=False)
dim_date.to_csv(data_processed / "dim_date.csv", index=False)
